In [28]:
import torch.optim as optim
import torch
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import random
import os
%run helpers_preproc.ipynb
%run alternate_data.ipynb

In [29]:
origDataDir = 'SHREC11/'
newDataDir = 'SHREC11_plus/'

In [30]:
#generate additional data and update label
origLabel = np.array(readLbl(600,origDataDir + 'labels.txt'),dtype = int) #first take in the original labels
newLabelFile = open(newDataDir + "labels.txt", "w")

In [31]:
radiuss = [0.0002,0.0004,0.0008,0.0016]

for i in range(600):
    #First create the new obj file
    origN = origDataDir + 'T' + str(i) + '.obj'
    
    for j in range(len(radiuss)):
        radius = radiuss[j]
        newN = newDataDir + 'T' + str(i+600*(j)) + '.obj'
    
        #newLabelFile.write("T" + str(i) + ".obj" + " " + str(origLabel[i]) + "\n")
        if not (write_new_files(origN,newN,radius)):
            print('Data generation failed on ' , i)
        
        newLabelFile.write("T" + str(i + 600*(j)) + ".obj" + " " + str(origLabel[i]) + "\n")

    
newLabelFile.close()

In [32]:
#read in the data, get normalized adjacency (NxN)
num_meshes = 600 * 4
train_size = 450 * 4
test_size = 150 * 4
mesh_dir = 'SHREC11_plus/'

In [33]:
num_swaps = 10000

labels = np.array(readLbl(num_meshes,mesh_dir+'labels.txt'))

for swap_idx in range(num_swaps):
    idxA = random.randint(0,num_meshes-1)
    idxB = random.randint(0,num_meshes-1)
    
    if idxA != idxB:
        labelA = labels[idxA]
        labelB = labels[idxB]
        labels[idxA] = labelB
        labels[idxB] = labelA

        #change filenames
        os.rename(mesh_dir+'T' + str(idxA) + '.obj',mesh_dir+'temp.obj')
        os.rename(mesh_dir+'T' + str(idxB) + '.obj',mesh_dir+'T' + str(idxA) + '.obj')
        os.rename(mesh_dir+'temp.obj',mesh_dir+'T' + str(idxB) + '.obj')

newLabelFile = open(newDataDir + "labels.txt", "w")
for i in range(num_meshes):
        newLabelFile.write("T" + str(i) + ".obj" + " " + str(labels[i]) + "\n")


newLabelFile.close()

In [34]:
label_np = np.array(readLbl(num_meshes,mesh_dir+'labels.txt'))

In [35]:
node_sigs_list = []
normed_adjMats_list = []

for i in range(num_meshes):
    fName = 'T' + str(i) + '.obj'
    adj_noscale_read = obj_2_adj_noscale(mesh_dir + fName)
    adj_scaled_read = obj_2_adj(mesh_dir + fName)

    if (adj_scaled_read.shape[0] < 252):
        adj_noscale = np.empty([252,252])
        adj_scaled = np.empty([252,252])
    else:
        adj_noscale = adj_noscale_read
        adj_scaled = adj_scaled_read

    adj_normalized = adj_noscale / np.reshape(np.sum(adj_noscale,axis = 1),[adj_noscale.shape[0],1])

    #node level signal extraction
    node_degs = np.sum(adj_noscale,axis = 0)
    node_neigh_max = np.max(adj_scaled,axis = 0)
    #node_neigh_min = np.min(adj_scaled,axis = 0)
    node_neigh_sum = np.sum(adj_scaled,axis = 0)
    node_neigh_mean = np.sum(adj_scaled,axis = 0) / node_degs

    node_sig = np.stack([node_degs,node_neigh_max,node_neigh_sum,node_neigh_mean],axis = 1)
    
    node_sigs_list.append(node_sig)
    normed_adjMats_list.append(adj_normalized)

In [36]:
normed_adjMats = np.nan_to_num(np.stack(normed_adjMats_list))
node_sigs = np.nan_to_num(np.stack(node_sigs_list))

In [37]:
#feature_mean = np.reshape(np.mean(node_sigs,axis = 1),[num_meshes,1,5])
#node_sigs = node_sigs - feature_mean 

In [38]:
node_sigs

array([[[8.        , 0.21602031, 0.93197489, 0.11649686],
        [6.        , 0.19737095, 1.00369017, 0.1672817 ],
        [5.        , 0.16103456, 0.44536618, 0.08907324],
        ...,
        [6.        , 0.12915873, 0.50090544, 0.08348424],
        [7.        , 0.19328788, 1.12109104, 0.16015586],
        [7.        , 0.2305088 , 1.24528015, 0.17789716]],

       [[5.        , 0.06210198, 0.23493933, 0.04698787],
        [5.        , 0.11466726, 0.3035657 , 0.06071314],
        [3.        , 0.0375809 , 0.10481666, 0.03493889],
        ...,
        [8.        , 0.10067814, 0.48646736, 0.06080842],
        [6.        , 0.12029223, 0.56660735, 0.09443456],
        [6.        , 0.09960374, 0.38744729, 0.06457455]],

       [[5.        , 0.11337228, 0.38616764, 0.07723353],
        [5.        , 0.08760865, 0.37376395, 0.07475279],
        [5.        , 0.10911618, 0.38778972, 0.07755794],
        ...,
        [6.        , 0.10598813, 0.50822867, 0.08470478],
        [5.        , 0.093775

In [39]:
normed_adjMats_tr = normed_adjMats[:train_size,:,:]
node_sigs_tr = node_sigs[:train_size,:,:]
label_np_tr = label_np[:train_size]

normed_adjMats_ts = normed_adjMats[train_size:,:,:]
node_sigs_ts = node_sigs[train_size:,:,:]
label_np_ts = label_np[train_size:]

label_mat_tr = torch.tensor(np.where(igl.all_pairs_distances(label_np_tr,label_np_tr,False) > 0.5,0,1)).float()
label_mat_ts = torch.tensor(np.where(igl.all_pairs_distances(label_np_ts,label_np_ts,False) > 0.5,0,1)).float()

In [40]:
normed_adjMats_tr = torch.tensor(normed_adjMats_tr).float()
node_sigs_tr = torch.tensor(node_sigs_tr).float()
label_tr = torch.tensor(label_np_tr).float()

normed_adjMats_ts = torch.tensor(normed_adjMats_ts).float()
node_sigs_ts = torch.tensor(node_sigs_ts).float()
label_ts = torch.tensor(label_np_ts).float()

In [41]:
#label_mat_train = form_label_matrix(label_mat, train_dict)

In [42]:
#normed_adjMats_list_train, node_sigs_list_train = prep_data(train_dict)

In [43]:
#normed_adjMats_train = np.stack(normed_adjMats_list_train)
#node_sigs_train = np.stack(node_sigs_list_train)

#normed_adjMats_train = torch.tensor(normed_adjMats_train,requires_grad=False).float()
#node_sigs_train = torch.tensor(node_sigs_train,requires_grad=False).float()

#normed_adjMats_train = torch.nan_to_num(normed_adjMats_train,0,0,0)
#node_sigs_train = torch.nan_to_num(node_sigs_train,0,0,0)

In [44]:
node_sigs_tr.shape

torch.Size([1800, 252, 4])

In [45]:
%run NNs.ipynb
g = GCN(4,[5,5,5,5])
n = MLP(252*5,[5,5,5,5])
output1 = g.forward(normed_adjMats_tr,node_sigs_tr)
output2 = n.forward(output1)
print(output1.shape)
print(output2.shape)

torch.Size([1800, 252, 5])
torch.Size([1800, 5])


In [49]:

def lossF(features,lbl_mat):
    disMat = torch.cdist(features,features)
    sameType = disMat * (lbl_mat-torch.diag(torch.ones(lbl_mat.shape[0])))
    diffType = disMat * (1-lbl_mat)

    diffTypeScaled = diffType * torch.exp(-diffType)
    diffTypeScaledMean = torch.sum(diffTypeScaled)/ torch.count_nonzero(diffTypeScaled)
    diffTypeMean = torch.sum(diffType)/ torch.count_nonzero(diffType)

    sameTypeMean = torch.sum(sameType) / torch.count_nonzero(sameType)
    sameTypeStd = torch.sum((sameType - sameTypeMean)**2) / torch.count_nonzero(sameType)

    toRet = sameTypeMean-diffTypeMean + 0.01 * torch.sqrt(sameTypeStd)
    #toRet = -(sameTypeMean-diffTypeMean)**2 #+ 0.5*(sameTypeMean + 0.01 * torch.sqrt(sameTypeStd))
    #toRet = -(sameTypeMean-diffTypeMean)**2 - diffTypeScaledMean + 0.5 * torch.sqrt(sameTypeStd)
    
    print(sameTypeMean.detach().numpy(),diffTypeMean.detach().numpy(),torch.sqrt(sameTypeStd).detach().numpy(), toRet.detach().numpy())
    return  toRet

In [50]:
optimizer = optim.Adam(g.weights + n.weights,lr = 0.0001)
for i in range(100000):
    optimizer.zero_grad()
    output = n.forward(g.forward(normed_adjMats_tr,node_sigs_tr))
    loss = lossF(output,label_mat_tr)
    loss.backward()
    optimizer.step()

0.00036976344 0.000403406 0.0021424582 -1.2217992e-05
0.0003708744 0.0004084153 0.002146539 -1.6075524e-05
0.0003767979 0.00041593498 0.0021936316 -1.720077e-05
0.0003819111 0.0004260396 0.00221118 -2.2016708e-05
0.00038851314 0.0004371264 0.0022480912 -2.6132351e-05
0.00039733248 0.00045090355 0.0022975104 -3.059597e-05
0.00040804257 0.00046751343 0.0023632138 -3.583872e-05
0.0004203311 0.00048621843 0.0024343808 -4.154351e-05
0.00043417065 0.00050720415 0.0025129593 -4.7903908e-05
0.00044995395 0.00053225714 0.0025990815 -5.6312376e-05
0.0004688024 0.00055932585 0.0027133534 -6.338992e-05
0.000487974 0.0005900347 0.0028153793 -7.390691e-05
0.00050966715 0.00062381185 0.002939007 -8.475463e-05
0.00053518714 0.0006607052 0.0030872521 -9.464555e-05
0.0005617541 0.0007011435 0.0032334395 -0.00010705502
0.0005901331 0.00074442575 0.0033931974 -0.0001203607
0.00062063156 0.00079173024 0.0035620604 -0.00013547808
0.00065482536 0.0008428088 0.003754455 -0.0001504389
0.0006911995 0.0008972579

KeyboardInterrupt: 

In [ ]:
output = n.forward(g.forward(normed_adjMats_tr,node_sigs_tr))
output.shape

In [ ]:
disMat = torch.cdist(output,output).flatten()
mask_same = torch.tensor(label_mat_tr - torch.diag(torch.ones(train_size)),dtype=bool).flatten()
mask_diff = torch.tensor(1 - label_mat_tr,dtype=bool).flatten()
sameComp = disMat[mask_same].detach().numpy()
diffComp = disMat[mask_diff].detach().numpy()
weightsSame = np.ones_like(sameComp) / len(sameComp)
weightsDiff = np.ones_like(diffComp) / len(diffComp)

In [ ]:
print(np.mean(sameComp),np.mean(diffComp))
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.hist(sameComp,bins=np.arange(0,np.max(sameComp),0.01),weights=weightsSame,log = False)
ax1.set_title('Same Comp')
ax2.hist(diffComp,bins=np.arange(0,np.max(diffComp),0.01),weights=weightsDiff,log = False)
ax2.set_title('Diff Comp')

In [ ]:
output = n.forward(g.forward(normed_adjMats_ts,node_sigs_ts))
output.shape

In [ ]:
disMat = torch.cdist(output,output).flatten()
mask_same = torch.tensor(label_mat_ts - torch.diag(torch.ones(test_size)),dtype=bool).flatten()
mask_diff = torch.tensor(1 - label_mat_ts,dtype=bool).flatten()
sameComp = disMat[mask_same].detach().numpy()
diffComp = disMat[mask_diff].detach().numpy()
weightsSame = np.ones_like(sameComp) / len(sameComp)
weightsDiff = np.ones_like(diffComp) / len(diffComp)

In [ ]:
print(np.mean(sameComp),np.mean(diffComp))
f, (ax1, ax2) = plt.subplots(1, 2, sharey=True)
ax1.hist(sameComp,bins=np.arange(0,np.max(sameComp),0.01),weights=weightsSame,log = False)
ax1.set_title('Same Comp')
ax2.hist(diffComp,bins=np.arange(0,np.max(diffComp),0.01),weights=weightsDiff,log = False)
ax2.set_title('Diff Comp')

In [ ]:
#from matplotlib.pyplot import figure
#figure(figsize=(10, 8), dpi=80)


In [ ]:
BigDiffIndices = np.where(disMat.detach().numpy().flatten() > 0.75)[0].tolist()

In [ ]:
len(BigDiffIndices)

In [ ]:
vizMat = np.zeros([30,30])

In [ ]:
#Separating 30 categories into less categories

# for index in BigDiffIndices:
#     row, col = divmod(index,test_size)
#     cat1, cat2 = int(label_np_ts[row]),int(label_np_ts[col])
#     if cat1 == cat2:
#         print(cat1, cat2)
#     vizMat[cat1,cat2] += 1
#     vizMat[cat2,cat1] += 1


In [ ]:
vizMat = vizMat / np.max(vizMat)

In [ ]:
plt.matshow(vizMat)

In [ ]:
features = output.detach().numpy()

In [ ]:
features.shape

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=3)
newX = pca.fit_transform(features)
np.sum(pca.explained_variance_ratio_)

In [ ]:
newX.shape

In [ ]:
indices_to_show = [0,1,2,3,4]

In [ ]:
dataPlot = []
for i in range(len(indices_to_show)):
    a = newX[np.where(label_np_ts == indices_to_show[i]),:]
    a = np.reshape(a,[a.shape[1],a.shape[2]])
    categories = np.ones([a.shape[0],1]) * indices_to_show[i]
    b = np.concatenate((a,categories),axis = 1)
    dataPlot.append(b)
    
dataPlot = np.concatenate(dataPlot,axis = 0)

In [ ]:
dataPlot.shape

In [ ]:
plt.clf()

In [ ]:
from mpl_toolkits.mplot3d import axes3d

fig = plt.figure()
ax = fig.add_subplot(projection='3d')

ax.scatter(dataPlot[:,0], dataPlot[:,1], dataPlot[:,2], label=dataPlot[:,3], c = dataPlot[:,3], cmap = 'viridis')
#ax.axis('off')
ax.axes.xaxis.set_ticklabels([])
ax.axes.yaxis.set_ticklabels([])
ax.axes.zaxis.set_ticklabels([])
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_title('[0,1,2,3,4]')

#plt.legend(loc="upper right")

plt.show()